### ClearMap parameter sweep

New users *must* use this notebook to determine the optimal parameters for cell detection on their images before processing any data. Follow the instructions below

#### Warning #1: 
Check to make sure elastix is installed under `/usr/bin/elastix` on your local computer, as the `Settings.py` checks the Elastix paths before any imports. You can validate this by typing `which elastix` into the terminal to make sure the path is indeed `/usr/bin/elastix`.

#### Run the following imports:

In [4]:
import os, sys, shutil, pickle, tifffile, numpy as np
from xvfbwrapper import Xvfb; vdisplay = Xvfb(); vdisplay.start()
from itertools import product
from skimage.exposure import rescale_intensity

from ClearMap.cluster.preprocessing import updateparams, listdirfull, arrayjob, makedir, removedir
from ClearMap.cluster.directorydeterminer import directorydeterminer
from ClearMap.cluster.par_tools import celldetection_operations
from ClearMap.cluster.utils import load_kwargs

Set the home directory of your computer to the variable `systemdirectory` so appropriate file paths can be set in the rest of the dictionary.

In [2]:
systemdirectory = "/home/wanglab"

#### Set paths to data
The `inputdictionary` structure is as follows:

* Key = Path to data 
* Value = `list["xx", "##"]` where xx = regch, injch, or cellch and ## = two digit channel number

The two digit channel number is based on the order of channels imaged in the LaVision acquisition setup. This will typically be alphanumerically ordered. For example, if you acquired a multi-channel stack using the 488 and 647 nm laser, `488 = 00`, and `647 = 01`. 

The naming of the channels is categoried as follows:

* "regch" = channel to be used for registration, assumption is all other channels are signal
* "cellch" = channel(s) to apply cell detection
* "injch" = channels(s) to quantify injection site

If you have acquired multiple channels in different imaging sessions of the same sample, and thus have multiple folders, you can set them in the `inputdictionary` as such:
`inputdictionary={path_1: [["regch", "00"]], path_2: [["cellch", "00"], ["injch", "01"]]}`

In [3]:
inputdictionary={
os.path.join(systemdirectory, "LightSheetTransfer/Jess/202001_tpham_crus1_ymaze_cfos/200123_tpcrus1_lat_cfos_an13_1_3x_488_008na_1hfds_z10um_100msec_12-23-02"): [["regch", "00"]],
os.path.join(systemdirectory, "LightSheetTransfer/Jess/202001_tpham_crus1_ymaze_cfos/200123_tpcrus1_lat_cfos_an13_1_3x_647_008na_1hfds_z10um_250msec_12-11-38"): [["cellch", "00"]]
}

#### Filling the parameter dictionary

The parameter dictionary is used in the rest of the software to refer back to inputs and outputs, as well as imaging and processing attributes. The keys in the `params` dictionary represent:

* `inputdirectory` = the input dictionary we previously made. You can set the value to this `inputdictionary` variable.
* `outputdirectory` = where you want to save your parameter sweep results. Typically in the experiment's folder, based on the sample name.
* `xyz_scale` = the pixel size of your images. For the LaVision UltraMicroscope, this is typically: 1.3xobjective w/ 1xzoom = 5um/pixel; 4x objective = 1.63um/pixel
* `tiling_overlap` = percent overlap taken during tiling. Can set to "0" if no tiling was done.
* `blendtype` = process by which you want to blend the 2 lightsheets, typically "sigmoidal" is best.
